### 处理新增的bot数据

In [2]:
import pandas as pd

In [5]:
path = 'data/Result_27.csv'
path_20220617 = 'data/Result_3_202206017.csv'

dfs = pd.read_csv(path_20220617, sep=',', names=['zuber_id', 'intent_id', 'intent_name', 'corpus_id', 'text', 'synonyms'])
dfs.head()

,zuber_id,intent_id,intent_name,corpus_id,text,synonyms
0,1000000001,815860776309075968,6IB/推进-FAB-FA-AS,854654812441194496,我们可以清晰的可以清晰的可以看到，然后一周呢，能够快速的缓解，缓缓解患者的这个背痛，然后四周...,NaN
1,1000000001,815860776309075968,6IB/推进-FAB-FA-AS,854654812441194497,可申请可以快速控制炎症，缓解中毒症状，一周缓解背痛四周，缓解晨僵和疲劳，那么快速的缓解晨僵、...,NaN
2,1000000001,815860776309075968,6IB/推进-FAB-FA-AS,854654812441194498,可善挺可以快速的空盐，缓解中轴的症状呢我们来看一下可善挺的数据，那客栈挺一周就可以缓解背痛四...,NaN
3,1000000001,815860776309075968,6IB/推进-FAB-FA-AS,854654812441194499,可善挺治疗后一年缓解背痛，四周改善陈江和疲劳在一周内能够快速的缓解他的悲痛，在一个月内的时间...,NaN
4,1000000001,815860776309075968,6IB/推进-FAB-FA-AS,854654812441194500,可身体快速控源，缓解中轴症状，一周即可以缓解背痛四周，缓解晨僵和疲劳。那么咳嗽呢？通过研究结...,NaN


In [6]:
theme_binary = ["中行", "费卡", "6IB", "催收", '赛诺菲', '来优时', '推进']
theme_label = ['催收','费卡', '6IB', '推进']

def get_category(intent_name, theme):
    for t in theme:
        if t in intent_name:
            return t

dfs = dfs.dropna(subset=['text'])
dfs['category'] = dfs['intent_name'].apply(lambda x: get_category(x, theme_binary))
dfs['category_label'] = dfs['intent_name'].apply(lambda x: get_category(x, theme_label))
dfs_binary = dfs.dropna(subset=['category'])
dfs_label = dfs.dropna(subset=['category_label'])
dfs_binary = dfs_binary.dropna(subset=['text'])
dfs_label = dfs_label.dropna(subset=['text'])

In [7]:
## 查看intent_name 的总数 -》 考虑转换成标签后存在的总数
from sklearn import preprocessing

names = dfs_label['intent_name'].drop_duplicates().tolist()
print(f'names length: {len(names)}')
names = [(name,) for name in names]
mle = preprocessing.MultiLabelBinarizer()
mle.fit(y=names)
# print(mle.classes_)
for name in names:
    mle.transform([name])
print(mle.transform([('6IB/推进-FAB-B患者', '6IB-缔结-跟进计划',)]))

dfs_temp1= dfs_label[['intent_id', 'intent_name']]
dfs_temp2 = dfs_label[['intent_id', 'text']]
dfs_temp3 = dfs_label[['intent_id', 'category_label']]

intentid2name = dict(zip(dfs_temp1['intent_id'], dfs_temp1['intent_name']))
intentid2text = dict(zip(dfs_temp2['intent_id'], dfs_temp2['text']))
intentid2category = dict(zip(dfs_temp3['intent_id'], dfs_temp3['category_label']))
corpusid2text = dict(zip(dfs_label['corpus_id'], dfs['text']))
intent_ids = dfs_label['intent_id'].drop_duplicates().tolist()
intentid2corpus_id = {}
for intent_id in intent_ids:
    intentid2corpus_id[intent_id] = dfs_label[dfs_label['intent_id'] == intent_id]['corpus_id'].tolist()

names length: 84
[[0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]]


#### 读取线上bot的节点意图结构

In [8]:
import pandas as pd

node_path = 'data/tenant_bot_sale_cb_node.csv'
column_names = ['node', 'intents']

dfs_node = pd.read_csv(node_path, names=column_names)

category_counter = []

intents_tables = dfs_node['intents'].tolist()
intents_table_filter = []
for table in intents_tables:
    table_list = eval(table)
    if len(table_list) > 1:
        status = True
        temp_table = []
        for i in table_list:
            if i in intentid2name:
                category_counter.append(intentid2category[i])
                temp_table.append(i)
        if len(temp_table) > 1:
            intents_table_filter.append(temp_table)
print(len(intents_table_filter))
print(intents_table_filter[:3])

559
[[815860776309075968, 815860777558978560], [815860777999380480, 815860778305564672, 815860778603360256], [815860778603360256, 815860778909544448]]


In [10]:
import random

multi_label_data = []


for intent_table in intents_table_filter:
    sample_number = sum([intentid2corpus_id.__len__() for ii in intent_table]) // (len(intent_table) * 2)
    cur_sample = []
    while len(cur_sample) < sample_number:
        text = ''
        cur_corpus_id = []
        table_label = []
        for ii in intent_table:
            i = random.choice(intentid2corpus_id[ii])
            if len(text) + len(corpusid2text[i]) < 225:
                text += corpusid2text[i]
                table_label.append(intentid2name[ii])
        if len(table_label) > 1:
            cur_intent_name = tuple(table_label)
            cur_one_hot_label = mle.transform([cur_intent_name])[0]
            cur_sample.append((text, intent_table, cur_intent_name, cur_one_hot_label))
    multi_label_data.extend(cur_sample)

print(f'total: {len(multi_label_data)}')

total: 23478


#### 保存多标签数据

In [11]:
dfs_label_corpus = pd.DataFrame(multi_label_data, columns=['text', 'intent_id', 'intent_name', 'one_hot_label'])

# dfs_label_corpus.to_csv('data/data_corpus_multi_label_20220617.csv', sep='\t', index=False)

In [12]:
dfs_single_label = dfs_label[['intent_id', 'intent_name', 'text']]

### 构造标签数据并保存

In [13]:
def transform_label(x):
    label= None
    if isinstance(x, str):
        label = mle.transform([(x,)])[0]
    if isinstance(x, list):
        label = mle.transform([tuple(x)])[0]
    return label


dfs_single_label['one_hot_label'] = dfs_single_label['intent_name'].apply(lambda x: transform_label(x))

<ipython-input-13-bc7345670b61>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_single_label['one_hot_label'] = dfs_single_label['intent_name'].apply(lambda x: transform_label(x))


In [14]:
print(dfs_single_label.head(2))

print("****" * 10)
print(dfs_label_corpus.head(2))


            intent_id       intent_name  \
0  815860776309075968  6IB/推进-FAB-FA-AS   
1  815860776309075968  6IB/推进-FAB-FA-AS   

                                                text  \
0  我们可以清晰的可以清晰的可以看到，然后一周呢，能够快速的缓解，缓缓解患者的这个背痛，然后四周...   
1  可申请可以快速控制炎症，缓解中毒症状，一周缓解背痛四周，缓解晨僵和疲劳，那么快速的缓解晨僵、...   

                                       one_hot_label  
0  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
****************************************
                                                text  \
0  隔扇体呢，可以达到一个快速控盐，缓解中轴症状的一个目标，它是可以一周缓解背痛四周，缓解晨僵四...   
1  就是科三呢，它有一个就是快速控盐，缓解中轴症状的一个优势，那他这边一周是可以缓解背痛四周，缓...   

                                  intent_id  \
0  [815860776309075968, 815860777558978560]   
1  [815860776309075968, 815860777558978560]   

                             intent_name  \
0  (6IB/推进-FAB-FA-AS, 6IB/推进-FAB-FA-PSO)   
1  (6IB/推进-FAB-FA-AS, 6IB/推进-FAB-FA-PSO)   

                                       one_hot_labe

### 合并label 数据，并保存

In [17]:
dfs_label_merge = pd.concat([dfs_single_label, dfs_label_corpus], ignore_index=False)

dfs_label_train = dfs_label_merge[['text', 'one_hot_label']]
print(dfs_label_train.shape)

dfs_label_train.to_csv('output_data/data_label_corpus_20220620_v2.csv', sep='\t', index=False)

(25987, 2)


### 保存处理好的数据-0515

In [109]:
dfs_binary = dfs_binary[['category','intent_id', 'intent_name', 'corpus_id', 'text']]


dfs_binary.to_csv('data/bot_origin_corpus_20220617.csv', sep='\t', index=False)

### 合并不同方法的数据

In [2]:
import pandas as pd

path1 = 'output_data/bot_multi_intent_train_20220517_3.csv'
path2 = 'output_data/bot_train_20220516_8_2.csv'
path3 = 'data/wz_train.csv'

df1 = pd.read_csv(path1, sep='\t')
df2 = pd.read_csv(path2, sep='\t')
df3 = pd.read_csv(path3, sep='\t',names=['id', 'query', 'candidate', 'label'])

In [3]:
df3 = df3[['query', 'candidate', 'label']]

dfs = pd.concat([df1, df2, df3], ignore_index=True)

dfs.to_csv('output_data/bot_corpus_20220517_v2.csv', sep='\t', index=False)

### 保存处理好的数据-0612

In [1]:
import pandas as pd

path1 = 'output_data/bot_multi_intent_train_20220517_3.csv'
path2 = 'output_data/bot_train_20220612_v2_50_1.csv'
path3 = 'data/wz_train.csv'

df1 = pd.read_csv(path1, sep='\t')
df2 = pd.read_csv(path2, sep='\t')
df3 = pd.read_csv(path3, sep='\t',names=['id', 'query', 'candidate', 'label'])

In [3]:
df3 = df3.iloc[:50000]

In [4]:
df3 = df3[['query', 'candidate', 'label']]

dfs = pd.concat([df1, df2, df3], ignore_index=True)

dfs.to_csv('output_data/bot_corpus_20220612_v4.csv', sep='\t', index=False)